## Radon-Cumulative Distribution Transform Nearest Subspace (RCDT-NS) Classifier

This demo code will demonstrate how to use the RCDT-NS classifier from the *PyTransKit* package.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To use the classifer, users need to install 'pytranskit' package using
'pip install pytranskit'.
This package can also be cloned/downloaded from github link
'https://github.com/rohdelab/PyTransKit.git', in that case users need to
specify the directory to 'pytranskit' using sys.path.append(/path/to/pytranskit)
in the codes.

## Steps:
The following example will demonstrate how to:
* create and initialize an instance of the class RCDT_NS
* train the model with training images
* apply the model to predict calss labels of the test images
In this example we have used MNIST dataset stored in the *data* folder

## Preliminaries
### 1. Import necessary python libraries

In [20]:
import shutil

folder_path = "/content/EMNIST"  # Replace with your folder path

shutil.rmtree(folder_path)


In [47]:
from google.colab import drive
import zipfile
import os


# Define the path to the ZIP file in Drive
zip_path = "/content/drive/MyDrive/cse499b/fingerprint.zip"  # Change this to your ZIP file path
extract_path = "/content/fingerprint"  # Change this to your desired extraction location

# Create extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete! Files are available at:", extract_path)


Extraction complete! Files are available at: /content/fingerprint


In [ ]:
!pip install pytranskit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys
sys.path.append('../')
from pytranskit.classification.utils import *

use_gpu = False

### 2. Import RCDT-NS class from *PyTransKit* package

In [ ]:
from pytranskit.classification.rcdt_ns import RCDT_NS

### 3. Load dataset
For loading data we have used *load_data* function from the *pytranskit/classifier/utils.py* script. It takes name and directory of the dataset, and total number of classes as input. Returns both train and test images in two separate 3d arrays of shape (n_samples, n_rows, n_columns), and corresponding class labels. User can use there own implementation to load data, just need to make sure that the output arrays are consistent.

In [50]:
# Load dataset

datadir = '/content/fingerprint'
dataset = 'fingerprint'
num_classes = 10          # total number of classes in the dataset
(x_train, y_train), (x_test, y_test) = load_data(dataset, num_classes, datadir)  # load_data function from utils.py

loading data from mat files


FileNotFoundError: [Errno 2] No such file or directory: '/content/fingerprint/fingerprint/training/dataORG_0.mat'

In this example we have used 512 randomly chosen samples per class to train the model. We have used another function *take_train_samples* function from *utils.py* script for this. User can use their own script.

In [43]:
# Choose a subset of training and test sets (optional)

n_samples_perclass = 512  # total number of training samples per class used in this example
x_train_sub, y_train_sub = take_train_samples(x_train, y_train, n_samples_perclass,
                                              num_classes, repeat=0) # function from utils.py

## Classification pipeline
### 1. Create an instance of RCDT_NS class
Now that we have training and testing sets at hand, we are ready to train and test the classifier. The first step is to create an instance of the RCDT_NS class. Users need to specify total number of classs (num_classes) and directions (optional) in degrees (theta) of the Radon projections.

In [32]:
# Create an instance of RCDT_NS class

theta = np.linspace(0, 176, 45)    # choose the angles in degrees that will be used to calculate Radon projections
rcdt_ns_obj = RCDT_NS(num_classes, theta, rm_edge=True)

### 2. Training phase

Let us assume, $\{s_1^{(k)}, s_2^{(k)}, \cdots \}, k=1,2,\cdots,N_{class},$ denotes a set of training images where $(k)$ denotes the class label.

The training algorithm includes the following steps: for each class $k$

1. Transform training samples using R-CDT to obtain $\left\{ \widehat{s}_1^{(k)}, \widehat{s}_2^{(k)}, \cdots  \right \}$

2. Orthogonalize $\left\{ \widehat{s}_1^{(k)}, \widehat{s}_2^{(k)}, \cdots  \right \}\cup\mathbb{U}_T$ to obtain the set of basis vectors $\left\{b_1^{(k)},b_2^{(k)},\cdots\right\}$. Here, $\mathbb{U}_T=\{u_1(t,\theta),u_2(t,\theta)\}$, with $u_1(t,\theta)=\cos\theta$ and $u_2(t,\theta)=\sin\theta$.

3. Use the output of orthogonalization procedure matrix $B^{(k)}$ that contains the basis vectors in its columns as follows:
\begin{equation}
    B^{(k)} = \begin{bmatrix}b^{(k)}_1, b^{(k)}_2, \cdots \end{bmatrix}\notag
\end{equation}


This function takes the train samples and labels as input, and stores the basis vectors for corresponding classes in a *private* variable. This variable will be used in the *predict* function in the test phase

In [44]:
# Training phase

rcdt_ns_obj.fit(x_train_sub, y_train_sub)


Calculating RCDTs for training images ...
Generating basis vectors for each class ...


### 3. Testing phase

Let us consider a testing image $s$ whose class is to be predicted by the classification model described above. As a first step, we apply R-CDT on $s$ to obtain the transform space representation $\widehat{s}$. The class of $\widehat{s}$ is then estimated to be
\begin{equation}
    \arg \min_k \| \widehat{s} - A^{(k)} \widehat{s} \|^2.\nonumber
\end{equation}
where, $A^{(k)}=B^{(k)}{B^{(k)}}^T$.

The *predict* function in the following takes the train samples as input and returns the predicted class labels

In [45]:
# Testing phase

preds = rcdt_ns_obj.predict(x_test, use_gpu)


Calculating RCDTs for testing images ...
Finding nearest subspace for each test sample ...


### 4. Display performance metrics
Test accuracy can be calculated and displayed as follows:

In [46]:
# Print test accuracy

print('\nTest accuracy: {}%'.format(100*accuracy_score(y_test, preds)))


Test accuracy: 8.98972602739726%
